## Rendimenti attesi impliciti

Cominciamo a ragionare su un nuovo modello che permette di lavorare con i dati ottenuti dai gestori attivi. cominciamo con l'introdurre l'estrazione delle stime implicite rendimenti attesi, il primo passo della procedura di Black Litterman.

Possiamo usare i benckmark come punto di ancoraggio, in modo tale che se non abbiamo opinioni attive il nostro portafoglio converga verso una soluzione ottimale.

Il modello che stiamo per costruire è una applicazione dell'analisi Bayesiana nella costruzione di portafogli. Questo modello è interessante perchè porta a costruire portafogli intuitivi.

Entriamo nella analisi di questo modello. La prima parte di questo processo parte dal presupposto che il mercato o il benchmark siano dei portafogli ottimali. Da questo passo applichiamo una sorta di **reverse engineer** per ottenere i pesi di partenza.

Come funziona questo **reverse engineering**?
Nella costruzione classica del portafoglio si inizia con una serie di parametri (rendimenti attesi, varianza, covarianza) e da li con i metodi di ottimizzazione del portafoglio si arriva ad una serie di pesi che rappresenta il nostro portafoglio ottimale.

Con il reverse engineering si parte da dalla matrice di covarianza e dai pesi del benchmark, da questo mediante l'ottimizzazione otteremo un rendimento atteso come risultato, questo viene chiamato anche **rendimento atteso implicito**.

Ora facciamo un esercizio di stile, supponiamo che il nostro gestore attivo ci dica che il valore di una azione salirà al 12% potremmo confrontarlo con il rendimento atteso implicito, supponiamo che otteremo il 10% allora potremmo dire che il mercato in questo momento è rialzista.

Alla fine tratteremo questo vettore di rendimenti impliciti $\Pi$ come punto di riferimento e gli aggiungeremo una sorta di incertezza $\epsilon^{e}$

Da questa stima a priori incorporeremo le prospettive dei gestori attivi al fine di ottenere una stima a posteriori che verrà  utilizzata per creare i pesi del nostro portafoglio.

## Incorporazione delle viste attive

Come detto prima il modello permette di mescolare i rendimenti impliciti del mercato con le aspettative del gestore attivo, da questi due dati otteremmo tramite un lavoro di statistica bayesiana un rendimento atteso come risultato.

Cerchiamo di capire come modellare queste viste attive, una vista di un portafoglio $P$ è modellata come una distribuzione normale con media $Q$ (un vettore) e deviazione standard $\Omega$.

Impostiamo $K$ come il numero di viste/opinioni che abbiamo e $N$ il numero totale di asset allora $K \leq N$ allora: 

- $P$ è una matrice $K \times N$ che identifica gli asset coinvolti nella vista.
- $Q$ sarà un vettore di dimensione $K$ con il rendimento atteso del nostro portafoglio.
- $\Omega$ è una matrice $K \times N$ che mostra i livelli di incertezza delle nostre viste.


### Applicazione del modello

Come nota da tenere a mente il vettore dei rendimenti $\Pi$ è altamente correlato con il modello CAPM, inoltre le voste possono essere espresse sia in termini assoluti che in termini relativi 

## Formule del modello

Assumiamo di avere $N$ asset e $K$ view. Ci sono due insiemi di parametri che servono al modello:

Il primo insieme di parametri è relativo al mercato 

\begin{array}{ll}
w & \mbox{Un vettore colonna ($N \times 1$) dei pesi di equilibrio del mercato} \\
\Sigma & \mbox{Una matrice di covarianza ($N \times N$) degli asset} \\
R_f & \mbox{Il tasso risk free} \\
\delta & \mbox{Avversione al rischio dell'investitore}  \\
\tau & \mbox{Uno scalare indicante l'incertezza della stima a priori (i dettagli qui sotto)}
\end{array}

Alcuni parametri possono essere dedotti se non specificati, Per esempio il parametro di avversione al rischio può essere impostato arbitrariamente Di solito viene usato un valore tra $\delta = 2.5$ e  $\delta = 2.14$.

$\tau$ può portare ad un po di confusione, alcuni lo impostano a 1, nel paper originale viene suggerito di usare un numero piccolo. Una tecnica comune è quella di usare $\tau = 1/T$ con $T$ il numero di perodi usati.

Ad esempio se useremo 5 periodi  $T=5$ imposteremo $1/5$ $\tau=.2$

Il secondo insieme di parametri rappresenta le stime degli investitori che sono espresse come:

\begin{array}{ll}
Q & \mbox{An $K \times 1$ ``Viste qualitative`` o semplicemente la matice delle viste} \\
P & \mbox{A $K \times N$ ``Proiezione`` il collegamento tra ogni vista e il so assett } \\
\Omega & \mbox{Una matrice di covarianza che rappresenta l'incertezza delle viste}
\end{array}

Le viste sono rappresentate in  $Q$ e $P$ nel seguente modo:

Se la $k$-esima vista è una vista assoluta viene rappresenta impostando l'elemento $Q_k$ del rendimento atteso e impostando $P_{ki}$ a 1 e tutti gli altri elementi della riga $k$ di $P$ a zero.

Se la $k$-esima vista è una vista relativa, tra l'asset $i$ e $j$ questo viene rappresentato impostando $Q_k$ con la differenza del rendimento $i$ and $j$ e impostanto $P_{ki}$ a $-1$ per l'asset che sottoperformante $P_{kj}$ a $+1$ e tutti gli altri elementi della riga $k$

$\Omega$ viene impostata con l'incertezza specifica o viene dedotta dall'utente o dai dati. Viene suggerito che sia una matrice diagonale ottenuta da  $P \tau \Sigma P^T$ ed è quello che faremo nei test iniziali.

#### La formula principale

Il primo passo della procedura è _l'ottimizzazione inversa_ che deduce il vettore dei rendimenti impliciti $\pi$ che deriva dai pesi in equilibrio del mercato $w$:

$$\pi = \delta\Sigma w$$

Poi, i rendimenti e le covarianze a posteriori sono ottenuti con la formula di  _Black-Litterman_ che viene estratta dalle seguenti equazioni:

\begin{equation}
\label{eq:blMuOrig}
\mu^{BL} = [(\tau\Sigma)^{-1} + P \Omega^{-1} P]^{-1}[(\tau\Sigma)^{-1} \pi + P \Omega^{-1} Q]
\end{equation}

\begin{equation}
\label{eq:blSigmaOrig}
\Sigma^{BL} = \Sigma + [(\tau\Sigma)^{-1} + P \Omega^{-1} P]^{-1}
\end{equation}

#### La matrice inversa $\Omega$

Le formule indicate nelle equazioni prima  sono facilemente implementabili tuttavia dobbiamo trattare $\Omega^{-1}$. Sfortunatamente $\Omega$ spesso non è invertibile e questo da parecchie difficoltà.

La formula è facilmente trasformabile in una equazione che non richiede questa inversione. 
Inoltre queste nuove formule risultano anche più stabili numericamente.

\begin{equation}
\label{eq:blMu}
\mu^{BL} = \pi + \tau \Sigma P^T[(P \tau \Sigma P^T) + \Omega]^{-1}[Q - P \pi]
\end{equation}

\begin{equation}
\label{eq:blSigma}
\Sigma^{BL} = \Sigma + \tau \Sigma - \tau\Sigma P^T(P \tau \Sigma P^T + \Omega)^{-1} P \tau \Sigma
\end{equation}

### Implementazione

Inziamo a vedere come implementare questo modello, confronteremo i risultati anche con i pesi ottenute dalla ottimizzazione classica al fime di vedere le differenze dei vari pesi.

La procedura di Black Litterman è implementata in python nella funzione `bl`. Prima di implementare il corpo di questa fuznione avremo bisogno di alcune funzioni di utilità.

Numpy tratta i vettori colonna differentemente dagli array monodimensionali. Al fine di ottenere un vettore colonna creiamo la funzione `as_colvec` che prende un array numpy e lo trasforma in in vettore colonna.

In [1]:
import numpy as np
import pandas as pd

def as_colvec(x):
    if (x.ndim > 1):
        return x
    else:
        return np.expand_dims(x, axis=1)

In [2]:
vec = np.arange(4)
vec

array([0, 1, 2, 3])

In [3]:
as_colvec(vec)

array([[0],
       [1],
       [2],
       [3]])

Il primo passo della procedura è il **reverse engineer** al fine di ottenere un vettore $\pi$ da un insieme di pesi $w$

In [4]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

Come detto precedentemente se l'investitore non ha una idea specifica per quantificare l'incertezza associata alla vista della matrice $\Omega$ possiamo semplificare l'assunzione che $\Omega$ sia proporzionale alla varianza della stima a priori.

$$\Omega = diag(P (\tau \Sigma) P^T) $$

In [5]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

Si può passare poi al calcolo del rendimento atteso a posteriori con la seguente funzione

In [6]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


Ora abbiamo tutto per iniziare a lavorare iniziamo con un caso semplice derivato da _Statistical Models and Methods for Financial Markets (Springer Texts in Statistics) 2008th Edition, Tze Lai and Haipeng Xing_.

Consideriamo un portafoglio con due azioni Intel (INTC) e Pfizer (PFE).
Dalla tabella 3.1 a pagina 72 del libro, otteniamo la matrice di covarianza (moltiplicata per $10^4$).

\begin{array}{lcc}
INTC & 46.0 & 1.06 \\
PFE   & 1.06 & 5.33
\end{array}


Assumiamo che Intel abbia una capitalizzazione di mercato pari a 80 Miliardi di Dollari e che Pfizer ne abbia 100 Miliardi di Dollari (I dati non sono perfetti ma servono come scopo dell'esercizio).

Se prendiamo la capitalizzazione come peso avremo che $W_{INTC} = 80/180 = 44\%, W_{PFE} = 100/180 = 56\%$.

Possiamo calcolare i rendimenti impliciti $\pi$ come:

In [7]:
tickers = ['INTC', 'PFE']
s = pd.DataFrame([[46.0, 1.06], [1.06, 5.33]], index=tickers, columns=tickers) *  10E-4
pi = implied_returns(delta=2.5, sigma=s, w=pd.Series([.44, .56], index=tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

Deduciamo che i rendimenti impliciti per INTC sono leggermente superiori al 5% mentre saranno leggermente inferiori all'1% per PFE.

Assumiamo che l'investore pensi che Intel avra un rendimento del 2\% e che Pfizer avrà un rendimento pari al 4\%. Esaminiamo i pesi ottimali usando la procedura di Markowitz.

Che cosa potrebbe succedere se usiamo questi rendimenti attesi nel portafoglio ottimizzato?

I pesi del portafoglio Max Sharpe Ratio (MSR) sono facilmente calcolabili nella forma implicita se non ci sono vincoli ai pesi.

$$ W_{MSR} = \frac{\Sigma^{-1}\mu_e}{\bf{1}^T \Sigma^{-1}\mu_e} $$


Con $\mu_e$  il vettore dei rendimenti attesi e $\Sigma$ la matrice di variance-covariance. 

In [8]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [9]:
mu_exp = pd.Series([.02, .04],index=tickers) # INTC and PFE
np.round(w_msr(s, mu_exp)*100, 2)

INTC     3.41
PFE     96.59
dtype: float64

Ecco che la procedura crea dei pesi non realistici con un 96% investito in Pfizer e nebi del 4\% in Intel. Questa è una soluzione non praticabile e nessun investore farebbe una scommessa così drammatica.

In contrasto a questi valori usiamo la procedura di Black Litterman calcolando automaticamente $\Omega$, attraverso questa procedura otteniamo i nuovi rendimenti attesi mescolando i rendimenti impliciti e le viste dell'investirore

In [10]:
# Absolute view 1: INTC will return 2%
# Absolute view 2: PFE will return 4%
q = pd.Series({'INTC': 0.02, 'PFE': 0.04})

# The Pick Matrix
# For View 2, it is for PFE
p = pd.DataFrame([
# For View 1, this is for INTC
    {'INTC': 1, 'PFE': 0},
# For View 2, it is for PFE
    {'INTC': 0, 'PFE': 1}
    ])

# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC':.44, 'PFE':.56}), sigma_prior=s, p=p, q=q)
# Black Litterman Implied Mu
bl_mu


INTC    0.037622
PFE     0.024111
dtype: float64

la domanda ora è questi valori portano a portafogli più realistici? per rispondere a questa domanda passiamo all'ottimizzatore i rendimenti attesi e la matrice di covarianza.

In [11]:
# Use the Black Litterman expected returns to get the Optimal Markowitz weights
w_msr(bl_sigma, bl_mu)

INTC    0.140692
PFE     0.859308
dtype: float64

Vediamo che i pesi sono più ragionati e vicini al punto di partenza 45/55.

### Un altro esempio una vista relativa

In questa parte esaminiamo le viste relative. Tenaimo sempre il nostro portafoglio con i rendimenti impliciti

In [12]:
# Expected returns inferred from the cap-weights
pi


INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

assumiamo ora che Intel supererà Pfizer  di un 2%, la vista implementata ora sarà:

In [13]:
q = pd.Series([
# Relative View 1: INTC will outperform PFE by 2%
  0.02
    ]
)
# The Pick Matrix
p = pd.DataFrame([
  # For View 1, this is for INTC outperforming PFE
  {'INTC': +1, 'PFE': -1}
])

# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC': .44, 'PFE': .56}), sigma_prior=s, p=p, q=q)
# Black Litterman Implied Mu
bl_mu

INTC    0.041374
PFE     0.009646
dtype: float64

Ancora una volta vediamo che i rendimenti a posteriori sono un mix tra i rendimenti impliciti e le viste degli investitori. La sovraperfomance di Intel nei rendimenti impliciti è:

In [14]:
pi[0]-pi[1]

0.043456

Al contrario l'investitore riteneva che sarebbe stato solo del 2% la procedura ha miscelato la vista del mercato e l'opinione dell'investore

In [15]:
bl_mu[0]-bl_mu[1]

0.031728

In [16]:
# Use the Black Litterman expected returns and covariance matrix
w_msr(bl_sigma, bl_mu)

INTC    0.347223
PFE     0.652777
dtype: float64

Questi sembrano ei pesi più che ragionabili e dimostrano la potenza di usare una procedura di Black Litterman. In contrasto se avessimo considerato la stessa cosa senza questo metodo avremmo ottenuto:



In [17]:
w_msr(s, [.03, .01])

INTC    0.258528
PFE     0.741472
dtype: float64

Questi pesi non sono giustificati data la visione debole in fatti se avessimo impostato dei valori con 2\% e 0\% i risultati sarebbero stati più estremi.



In [18]:
w_msr(s, [.02, .0])

INTC    1.248244
PFE    -0.248244
dtype: float64

## Riprodurre i risultati He-Litterman

Siamo pronti per riprodurre i risultati del paper originale He-Litterman, il documento originale coinvolgeva l'allocazione tra 7 nazioni.

In [19]:
# The 7 countries ...
countries  = ['AU', 'CA', 'FR', 'DE', 'JP', 'UK', 'US'] 
# Table 1 of the He-Litterman paper
# Correlation Matrix
rho = pd.DataFrame([
    [1.000,0.488,0.478,0.515,0.439,0.512,0.491],
    [0.488,1.000,0.664,0.655,0.310,0.608,0.779],
    [0.478,0.664,1.000,0.861,0.355,0.783,0.668],
    [0.515,0.655,0.861,1.000,0.354,0.777,0.653],
    [0.439,0.310,0.355,0.354,1.000,0.405,0.306],
    [0.512,0.608,0.783,0.777,0.405,1.000,0.652],
    [0.491,0.779,0.668,0.653,0.306,0.652,1.000]
], index=countries, columns=countries)

# Table 2 of the He-Litterman paper: volatilities
vols = pd.DataFrame([0.160,0.203,0.248,0.271,0.210,0.200,0.187],index=countries, columns=["vol"]) 
# Table 2 of the He-Litterman paper: cap-weights
w_eq = pd.DataFrame([0.016,0.022,0.052,0.055,0.116,0.124,0.615], index=countries, columns=["CapWeight"])
# Compute the Covariance Matrix
sigma_prior = vols.dot(vols.T) * rho
# Compute Pi and compare:
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq)
(pi*100).round(1)

AU    3.9
CA    6.9
FR    8.4
DE    9.0
JP    4.3
UK    6.8
US    7.6
Name: Implied Returns, dtype: float64

#### Vista 1: Germania contro il resto europa

Ora, imponiamo la nostra opinione che le azioni tedesche sovraperformeranno del 5% il resto delle azioni Europee.
Nella nostra tabella sono Francia e Regno Unito. Dividiamo che le sovraperformance sia proporzionale alla capitalizzazione di Mercato per Francia e Regno Unito.

In [20]:
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(countries), index=countries).T
# find the relative market caps of FR and UK to split the
# relative outperformance of DE ...
w_fr =  w_eq.loc["FR"]/(w_eq.loc["FR"]+w_eq.loc["UK"]).values
w_uk =  w_eq.loc["UK"]/(w_eq.loc["FR"]+w_eq.loc["UK"]).values

p.iloc[0]['DE'] = 1.
p.iloc[0]['FR'] = -w_fr
p.iloc[0]['UK'] = -w_uk
(p*100).round(1)

C:\Users\DalFornoF\AppData\Local\Temp\ipykernel_3444\3239996646.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  p.iloc[0]['FR'] = -w_fr
C:\Users\DalFornoF\AppData\Local\Temp\ipykernel_3444\3239996646.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  p.iloc[0]['UK'] = -w_uk


,AU,CA,FR,DE,JP,UK,US
0,0.0,0.0,-29.5,100.0,0.0,-70.5,0.0


Il risultato della implementazione $\mu^{BL}$

In [21]:
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)

AU     4.3
CA     7.6
FR     9.3
DE    11.0
JP     4.5
UK     7.0
US     8.1
dtype: float64

Il documento prosegue con il calcolo del portafoglio ottimale $w^*$

In [22]:
def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
(wstar*100).round(1)

AU     1.5
CA     2.1
FR    -4.0
DE    35.4
JP    11.0
UK    -9.5
US    58.6
dtype: float64

Calcoliamo infine  $w^* - \frac{w_{eq}}{1+\tau}$ che rappresenta la differenza dei pesi tra il portafoglio ottimale e il portafolgio in equilibrio

In [23]:
w_eq  = w_msr(delta*sigma_prior, pi, scale=False)
# Display the difference in Posterior and Prior weights
np.round(wstar - w_eq/(1+tau), 3)*100

AU     0.0
CA    -0.0
FR    -8.9
DE    30.2
JP     0.0
UK   -21.3
US     0.0
dtype: float64

### Seconda View Canada contro Stati Uniti

Nel secondo caso ipotizziamo che le azioni Canadesi sovraperformeranno quelle statunitensi del 3%.

In [24]:
view2 = pd.Series([.03], index=[1])
q = q._append(view2)
pick2 = pd.DataFrame([0.]*len(countries), index=countries, columns=[1]).T
p = p._append(pick2)
p.iloc[1]['CA']=+1
p.iloc[1]['US']=-1
np.round(p.T, 3)*100

,0,1
AU,0.0,0.0
CA,0.0,100.0
FR,-29.5,0.0
DE,100.0,0.0
JP,0.0,0.0
UK,-70.5,0.0
US,0.0,-100.0


Calcoliamo ora i pesi del portafoglio 

In [25]:
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau = tau)
np.round(bl_mu*100, 1)

AU     4.4
CA     8.7
FR     9.5
DE    11.2
JP     4.6
UK     7.0
US     7.5
dtype: float64

Come nel caso precedente calcoliamo  $w^* - \frac{w_{eq}}{1+\tau}$ 

In [26]:
w_eq  = w_msr(delta*sigma_prior, pi, scale=False)
# Display the difference in Posterior and Prior weights
np.round(wstar - w_eq/(1+tau), 3)*100

AU     0.0
CA    -0.0
FR    -8.9
DE    30.2
JP     0.0
UK   -21.3
US     0.0
dtype: float64